In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

#load data
X_full = pd.read_csv('/Users/chuwen/Desktop/kaggle/House_pricing/data/train.csv', index_col = 'Id')
X_test_full = pd.read_csv('/Users/chuwen/Desktop/kaggle/House_pricing/data/test.csv', index_col = 'Id')

X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()


In [5]:
X_train1 = pd.get_dummies(X_train)
X_valid1 = pd.get_dummies(X_valid)

X_train1, X_valid1 = X_train1.align(X_valid1, join='left', axis=1)
X_train1, X_test1 = X_train1.align(X_test, join='left', axis=1)




my_model_2 = XGBRegressor(n_estimators = 400, learning_rate = 0.05, n_jobs=4) # Your code here

# Fit the model
my_model_2.fit(X_train1, y_train)

# Get predictions
predictions_2 = my_model_2.predict(X_valid1) # Your code here

# Calculate MAE
mae_2 = mean_absolute_error(predictions_2, y_valid) # Your code here

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_2)

Mean Absolute Error: 16769.128478167808


In [6]:
# make prediction
X_transformed_test = pd.get_dummies(X_test)
X_transformed_full = pd.get_dummies(X_full[my_cols])
X_transformed_full, X_transformed_test = X_transformed_full.align(X_transformed_test, join='left', axis=1)


my_model_2.fit(X_transformed_full, y)
predictions_3 = my_model_2.predict(X_transformed_test)
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': predictions_3})
output.to_csv('submission_XGB.csv', index=False)
#!kaggle competitions submit -c home-data-for-ml-course -f submission_pipline.csv -m "Message"

In [7]:
!kaggle competitions submit -c home-data-for-ml-course -f submission_XGB.csv -m "Message"

100%|██████████████████████████████████████| 21.2k/21.2k [00:00<00:00, 48.8kB/s]
Successfully submitted to Housing Prices Competition for Kaggle Learn Users

# Make a simple pipeline

In [ ]:
numerical_transformer = SimpleImputer(strategy='constant')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

model = XGBRegressor()

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)])

pipeline.fit(X_train, y_train)

In [ ]:
predictions = pipeline.predict(X_valid)
mae_1 = mean_absolute_error(predictions, y_valid)
print('Mean absolute error:', mae_1)

# Tune the model

In [ ]:
model1 = XGBRegressor(n_estimator=400, learning_rate=0.05, n_jobs=4)
numerical_transformer = SimpleImputer(strategy='mean')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


pipeline1 = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model1)])

# X_valid_transformed = preprocessor.transform(X_valid)
# pipeline1.fit(X_train, y_train,
#              model__early_stopping_rounds=5,
#              model__eval_set=[(X_valid_transformed, y_valid)],
#              verbose=False)
pipeline1.fit(X_train, y_train)

In [ ]:
predictions2 = pipeline1.predict(X_valid)
mae_2 = mean_absolute_error(predictions2, y_valid)
print('Mean absolute error:', mae_2)